<a href="https://colab.research.google.com/github/4myLab/codestates/blob/main/session2/20220713.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [47]:
!pip install scipy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [1]:
import pandas as pd
train = pd.read_csv('https://ds-lecture-data.s3.ap-northeast-2.amazonaws.com/house-prices/house_prices_train.csv')

In [2]:
from sklearn.model_selection import train_test_split
train, test = train_test_split(train, test_size=260, random_state=2)

In [3]:
train.shape, test.shape

((1200, 81), (260, 81))

In [7]:
import numpy as np
import pandas as pd

In [9]:
def clean(df):
  df = df.copy()
  ## 타겟변수의 이상치(outlier)를 제거합니다.
  df['SalePrice'] = df[df['SalePrice'] < np.percentile(df['SalePrice'], 99)]['SalePrice']
  
  ## 타겟변수가 결측치인 샘플을 제거 합니다.
  df.dropna(subset=['SalePrice'], inplace=True)

  ## id 변수를 제거합니다.
  df = df.drop("Id", axis = 1)

  ## 변수를 합치고 새로운 변수를 생성합니다.(참고, https://www.kaggle.com/yadavsarthak/house-prices-advanced-regression-techniques/you-got-this-feature-engineering-and-lasso)
  df['All_Flr_SF'] = df['1stFlrSF'] + df['2ndFlrSF']
  df['All_Liv_SF'] = df['All_Flr_SF'] + df['LowQualFinSF'] + df['GrLivArea']
  df = df.drop(['1stFlrSF','2ndFlrSF','LowQualFinSF','GrLivArea'], axis=1)

  ## 변수의 이상치를 제거합니다.
  df['All_Flr_SF'] = df[df['All_Flr_SF'] < np.percentile(df['All_Flr_SF'], 99.5)]['All_Flr_SF']
  df['All_Liv_SF'] = df[df['All_Liv_SF'] < np.percentile(df['All_Liv_SF'], 99.5)]['All_Liv_SF']
  
  ## 결측값을 파이프라인에서 평균/중간값으로 처리하면 안되는 몇몇 특성이 있어 미리 처리하겠습니다.
  ## (https://www.kaggle.com/juliensiems/cleaning-new-features-gps-coordinates-included)
  
  ## 이외 특성들의 결측값은 파이프라인에서 처리합니다.
  
  # Alley : NA == "no alley access"
  df.loc[:, "Alley"] = df.loc[:, "Alley"].fillna("None")
  
  # BedroomAbvGr : NA ~ 0
  df.loc[:, "BedroomAbvGr"] = df.loc[:, "BedroomAbvGr"].fillna(0)
  
  # BsmtQual etc : NA == "no basement"
  df.loc[:, "BsmtQual"] = df.loc[:, "BsmtQual"].fillna("No")
  df.loc[:, "BsmtCond"] = df.loc[:, "BsmtCond"].fillna("No")
  df.loc[:, "BsmtExposure"] = df.loc[:, "BsmtExposure"].fillna("No")
  df.loc[:, "BsmtFinType1"] = df.loc[:, "BsmtFinType1"].fillna("No")
  df.loc[:, "BsmtFinType2"] = df.loc[:, "BsmtFinType2"].fillna("No")
  df.loc[:, "BsmtFullBath"] = df.loc[:, "BsmtFullBath"].fillna(0)
  df.loc[:, "BsmtHalfBath"] = df.loc[:, "BsmtHalfBath"].fillna(0)
  df.loc[:, "BsmtUnfSF"] = df.loc[:, "BsmtUnfSF"].fillna(0)
  
  # CentralAir : NA ~ No
  df.loc[:, "CentralAir"] = df.loc[:, "CentralAir"].fillna("N")
  
  # Condition : NA ~ Normal
  df.loc[:, "Condition1"] = df.loc[:, "Condition1"].fillna("Norm")
  df.loc[:, "Condition2"] = df.loc[:, "Condition2"].fillna("Norm")
  
  # EnclosedPorch : NA ~ no enclosed porch
  df.loc[:, "EnclosedPorch"] = df.loc[:, "EnclosedPorch"].fillna(0)
  
  # External stuff : NA ~ average
  df.loc[:, "ExterCond"] = df.loc[:, "ExterCond"].fillna("TA")
  df.loc[:, "ExterQual"] = df.loc[:, "ExterQual"].fillna("TA")
  
  # Fence : NA == "no fence"
  df.loc[:, "Fence"] = df.loc[:, "Fence"].fillna("No")
  
  # FireplaceQu : NA == "no fireplace"
  df.loc[:, "FireplaceQu"] = df.loc[:, "FireplaceQu"].fillna("No")
  df.loc[:, "Fireplaces"] = df.loc[:, "Fireplaces"].fillna(0)
  
  # Functional : NA == "typical"
  df.loc[:, "Functional"] = df.loc[:, "Functional"].fillna("Typ")
  
  # GarageType etc : NA == "no garage"
  df.loc[:, "GarageType"] = df.loc[:, "GarageType"].fillna("No")
  df.loc[:, "GarageFinish"] = df.loc[:, "GarageFinish"].fillna("No")
  df.loc[:, "GarageQual"] = df.loc[:, "GarageQual"].fillna("No")
  df.loc[:, "GarageCond"] = df.loc[:, "GarageCond"].fillna("No")
  df.loc[:, "GarageArea"] = df.loc[:, "GarageArea"].fillna(0)
  df.loc[:, "GarageCars"] = df.loc[:, "GarageCars"].fillna(0)
  
  # HalfBath : NA ~ no half baths above grade
  df.loc[:, "HalfBath"] = df.loc[:, "HalfBath"].fillna(0)
  
  # HeatingQC : NA ~ typical
  df.loc[:, "HeatingQC"] = df.loc[:, "HeatingQC"].fillna("TA")
  
  # KitchenAbvGr : NA ~ 0
  df.loc[:, "KitchenAbvGr"] = df.loc[:, "KitchenAbvGr"].fillna(0)
  
  # KitchenQual : NA ~ typical
  df.loc[:, "KitchenQual"] = df.loc[:, "KitchenQual"].fillna("TA")
  
  # LotFrontage : NA ~ no lot frontage
  df.loc[:, "LotFrontage"] = df.loc[:, "LotFrontage"].fillna(0)
  
  # LotShape : NA ~ regular
  df.loc[:, "LotShape"] = df.loc[:, "LotShape"].fillna("Reg")
  
  # MasVnrType : NA ~ no veneer
  df.loc[:, "MasVnrType"] = df.loc[:, "MasVnrType"].fillna("None")
  df.loc[:, "MasVnrArea"] = df.loc[:, "MasVnrArea"].fillna(0)
  
  # MiscFeature : NA == "no misc feature"
  df.loc[:, "MiscFeature"] = df.loc[:, "MiscFeature"].fillna("No")
  df.loc[:, "MiscVal"] = df.loc[:, "MiscVal"].fillna(0)
  
  # OpenPorchSF : NA ~ no open porch
  df.loc[:, "OpenPorchSF"] = df.loc[:, "OpenPorchSF"].fillna(0)
  
  # PavedDrive : NA ~ not paved
  df.loc[:, "PavedDrive"] = df.loc[:, "PavedDrive"].fillna("N")
  
  # PoolQC : NA == "no pool"
  df.loc[:, "PoolQC"] = df.loc[:, "PoolQC"].fillna("No")
  df.loc[:, "PoolArea"] = df.loc[:, "PoolArea"].fillna(0)
  
  # SaleCondition : NA ~ normal sale
  df.loc[:, "SaleCondition"] = df.loc[:, "SaleCondition"].fillna("Normal")
  
  # ScreenPorch : NA ~ no screen porch
  df.loc[:, "ScreenPorch"] = df.loc[:, "ScreenPorch"].fillna(0)
  
  # TotRmsAbvGrd : NA ~ 0
  df.loc[:, "TotRmsAbvGrd"] = df.loc[:, "TotRmsAbvGrd"].fillna(0)
  
  # Utilities : NA ~ all public utilities
  df.loc[:, "Utilities"] = df.loc[:, "Utilities"].fillna("AllPub")
  
  # WoodDeckSF : NA ~ no wood deck
  df.loc[:, "WoodDeckSF"] = df.loc[:, "WoodDeckSF"].fillna(0)
  df[df['Neighborhood'] == 'Greens']
  

  return df

In [10]:
train = clean(train)
test = clean(test)

In [11]:
train.shape, test.shape

((1188, 78), (257, 78))

In [15]:
from category_encoders import OneHotEncoder
from sklearn.feature_selection import f_regression, SelectKBest
from sklearn.impute import SimpleImputer
from sklearn.linear_model import Ridge
from sklearn.model_selection import cross_val_score
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler

np.seterr(divide='ignore', invalid='ignore')

target = 'SalePrice'
features = train.columns.drop([target])
X_train = train[features]
y_train = train[target]

X_test = test[features]
y_test = test[target]

pipe = make_pipeline(
    OneHotEncoder(use_cat_names = True),
    SimpleImputer(strategy='mean'),
    StandardScaler(),
    SelectKBest(f_regression, k=20),
    Ridge(alpha=1.0)
)

k = 3
scores = cross_val_score(pipe, X_train, y_train, cv=k, scoring='neg_mean_absolute_error')

print(f'MAE ({k} folds):', -scores)

/usr/local/lib/python3.7/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


MAE (3 folds): [19912.3716215  23214.74205495 18656.29713167]


In [16]:
-scores.mean()

20594.470269371846

In [18]:
scores.std()

1922.4635156881643

In [22]:
from category_encoders import TargetEncoder
from sklearn.ensemble import RandomForestRegressor

pipe = make_pipeline(
    TargetEncoder(min_samples_leaf=1, smoothing=1),
    SimpleImputer(strategy = 'median'),
    RandomForestRegressor(max_depth=10, n_jobs=-1, random_state=2)
)

k = 3

scores = cross_val_score(pipe, X_train, y_train, cv=k, scoring='neg_mean_absolute_error')

print(f'MAE for {k} folds:', -scores)

/usr/local/lib/python3.7/dist-packages/category_encoders/target_encoder.py:94: FutureWarning: Default parameter min_samples_leaf will change in version 2.6.See https://github.com/scikit-learn-contrib/category_encoders/issues/327
  category=FutureWarning)
/usr/local/lib/python3.7/dist-packages/category_encoders/target_encoder.py:99: FutureWarning: Default parameter smoothing will change in version 2.6.See https://github.com/scikit-learn-contrib/category_encoders/issues/327
  category=FutureWarning)
/usr/local/lib/python3.7/dist-packages/category_encoders/target_encoder.py:94: FutureWarning: Default parameter min_samples_leaf will change in version 2.6.See https://github.com/scikit-learn-contrib/category_encoders/issues/327
  category=FutureWarning)
/usr/local/lib/python3.7/dist-packages/category_encoders/target_encoder.py:99: FutureWarning: Default parameter smoothing will change in version 2.6.See https://github.com/scikit-learn-contrib/category_encoders/issues/327
  category=FutureWar

MAE for 3 folds: [16289.34502313 19492.01218055 15273.23000751]


In [23]:
-scores.mean()

17018.19573706592

In [24]:
scores.std()

1797.7668143704145

In [25]:
X_train.describe(exclude='number')

,MSZoning,Street,Alley,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,Condition1,...,GarageType,GarageFinish,GarageQual,GarageCond,PavedDrive,PoolQC,Fence,MiscFeature,SaleType,SaleCondition
count,1188,1188,1188,1188,1188,1188,1188,1188,1188,1188,...,1188,1188,1188,1188,1188,1188,1188,1188,1188,1188
unique,5,2,3,4,4,2,5,3,25,9,...,7,4,6,6,3,3,5,5,9,6
top,RL,Pave,None,Reg,Lvl,AllPub,Inside,Gtl,NAmes,Norm,...,Attchd,Unf,TA,TA,Y,No,No,No,WD,Normal
freq,936,1184,1112,759,1068,1187,856,1122,183,1024,...,709,508,1068,1080,1084,1184,961,1140,1036,986


In [26]:
enc = TargetEncoder(min_samples_leaf=1, smoothing=1000)
enc.fit_transform(X_train, y_train)['LotShape'].value_counts()

/usr/local/lib/python3.7/dist-packages/category_encoders/target_encoder.py:94: FutureWarning: Default parameter min_samples_leaf will change in version 2.6.See https://github.com/scikit-learn-contrib/category_encoders/issues/327
  category=FutureWarning)


166368.017847    759
190133.621187    391
194093.059489     28
196089.739213     10
Name: LotShape, dtype: int64

In [27]:
X_train['LotShape'].value_counts()

Reg    759
IR1    391
IR2     28
IR3     10
Name: LotShape, dtype: int64

In [29]:
y_train[X_train[X_train.LotShape == 'Reg'].index].sum() /759

161871.9486166008

In [36]:
import matplotlib.pyplot as plt
from category_encoders import OrdinalEncoder
from sklearn.model_selection import validation_curve
from sklearn.tree import DecisionTreeRegressor

pipe = make_pipeline(
    OrdinalEncoder(),
    SimpleImputer(),
    DecisionTreeRegressor()
)

depth = range(1, 30, 2)

ts, vs = validation_curve(
    pipe, X_train, y_train,
    param_name = 'decisiontreeregressor__max_depth',
    param_range = depth,
    scoring='neg_mean_absolute_error',
    cv=3,
    n_jobs=-1
)

train_scores_mean = np.mean(-ts, axis=1)
validation_scores_mean = np.mean(-vs, axis=1)

fig, ax = plt.subplots()
fig.dpi = 100
ax.plot(depth, train_scores_mean, label='training error')
ax.plot(depth, validation_scores_mean, label='validation error')
ax.vlines(5, 0, train_scores_mean.max(), color='blue')
ax.set(title='Validation Curve', xlabel='Model Complexity(max_depth)', ylabel='MAE')
ax.legend()
plt.show()



In [42]:
from sklearn.model_selection import RandomizedSearchCV

pipe = make_pipeline(
    OneHotEncoder(use_cat_names=True),
    SimpleImputer(),
    StandardScaler(),
    SelectKBest(f_regression),
    Ridge()
)

dists = {
    'simpleimputer__strategy':['mean','median'],
    'selectkbest__k':range(1, len(X_train.columns)+1),
    'ridge__alpha':[0.1,1,10]
}
clf = RandomizedSearchCV(
    pipe, 
    param_distributions=dists,
    n_iter=50,
    cv=3,
    scoring='neg_mean_absolute_error',
    verbose=1,
    n_jobs=-1
)

clf.fit(X_train, y_train)

Fitting 3 folds for each of 50 candidates, totalling 150 fits


RandomizedSearchCV(cv=3,
                   estimator=Pipeline(steps=[('onehotencoder',
                                              OneHotEncoder(use_cat_names=True)),
                                             ('simpleimputer', SimpleImputer()),
                                             ('standardscaler',
                                              StandardScaler()),
                                             ('selectkbest',
                                              SelectKBest(score_func=<function f_regression at 0x7fe5a5db7c20>)),
                                             ('ridge', Ridge())]),
                   n_iter=50, n_jobs=-1,
                   param_distributions={'ridge__alpha': [0.1, 1, 10],
                                        'selectkbest__k': range(1, 78),
                                        'simpleimputer__strategy': ['mean',
                                                                    'median']},
                   scoring='neg_mean_ab

In [43]:
print('최적 하이퍼파라미터: ', clf.best_params_)
print('MAE: ', -clf.best_score_)

최적 하이퍼파라미터:  {'simpleimputer__strategy': 'median', 'selectkbest__k': 76, 'ridge__alpha': 10}
MAE:  18444.101364661263


In [44]:
2 * len(X_train.columns) * 3 * 3

1386

In [45]:
pd.DataFrame(clf.cv_results_).sort_values(by='rank_test_score').T

,16,22,37,36,20,40,33,4,45,48,...,0,17,9,15,10,1,12,46,28,25
mean_fit_time,1.877026,1.926283,1.707941,1.524537,2.100795,1.039001,1.604131,1.026879,1.257721,1.294473,...,1.034672,1.370211,1.003403,1.712714,1.305577,0.97922,1.032176,1.298103,1.31465,1.932941
std_fit_time,0.358902,0.168431,0.193268,0.101556,0.151071,0.072892,0.286554,0.017836,0.166169,0.04565,...,0.020174,0.082644,0.018813,0.536235,0.393935,0.01844,0.00995,0.106836,0.265622,0.304078
mean_score_time,0.362245,0.404336,0.414145,0.320409,0.511415,0.237139,0.348618,0.252457,0.289225,0.365488,...,0.248886,0.457021,0.25966,0.305856,0.453473,0.251853,0.254539,0.363887,0.338972,0.607315
std_score_time,0.048858,0.037613,0.008064,0.035665,0.092976,0.002948,0.060612,0.008611,0.02612,0.022884,...,0.007931,0.11165,0.002971,0.014977,0.297083,0.003089,0.003846,0.053478,0.061596,0.206582
param_simpleimputer__strategy,median,mean,median,median,mean,median,mean,median,median,median,...,median,mean,median,median,mean,mean,median,median,median,median
param_selectkbest__k,76,62,58,54,61,74,63,64,50,73,...,16,12,12,14,11,8,7,5,4,2
param_ridge__alpha,10,10,10,10,1,1,1,1,10,0.1,...,1,1,0.1,10,0.1,10,10,0.1,1,1
params,"{'simpleimputer__strategy': 'median', 'selectk...","{'simpleimputer__strategy': 'mean', 'selectkbe...","{'simpleimputer__strategy': 'median', 'selectk...","{'simpleimputer__strategy': 'median', 'selectk...","{'simpleimputer__strategy': 'mean', 'selectkbe...","{'simpleimputer__strategy': 'median', 'selectk...","{'simpleimputer__strategy': 'mean', 'selectkbe...","{'simpleimputer__strategy': 'median', 'selectk...","{'simpleimputer__strategy': 'median', 'selectk...","{'simpleimputer__strategy': 'median', 'selectk...",...,"{'simpleimputer__strategy': 'median', 'selectk...","{'simpleimputer__strategy': 'mean', 'selectkbe...","{'simpleimputer__strategy': 'median', 'selectk...","{'simpleimputer__strategy': 'median', 'selectk...","{'simpleimputer__strategy': 'mean', 'selectkbe...","{'simpleimputer__strategy': 'mean', 'selectkbe...","{'simpleimputer__strategy': 'median', 'selectk...","{'simpleimputer__strategy': 'median', 'selectk...","{'simpleimputer__strategy': 'median', 'selectk...","{'simpleimputer__strategy': 'median', 'selectk..."
split0_test_score,-18102.653952,-18498.974404,-18771.46224,-18781.412397,-18864.337092,-18097.93614,-18552.133061,-18696.792397,-18887.49842,-18252.155108,...,-21291.840816,-21283.750278,-21284.603378,-21335.224209,-21887.350086,-22494.082426,-22277.720362,-23701.567257,-24012.673418,-25361.905851
split1_test_score,-19970.301239,-19970.862837,-19762.496242,-20014.960947,-19923.009973,-20187.609244,-20377.091997,-20379.174404,-20149.993181,-20187.928053,...,-23850.734157,-24702.871064,-24717.664245,-24611.525256,-24623.34042,-24733.276714,-25998.699707,-27930.516284,-28487.224723,-29950.679428


In [54]:
from scipy.stats import randint, uniform

pipe = make_pipeline(
    TargetEncoder(), 
    SimpleImputer(), 
    RandomForestRegressor(random_state=2)
)

dists = {
    'targetencoder__smoothing': [2.,20.,50.,60.,100.,500.,1000.], # int로 넣으면 error(bug)
    'targetencoder__min_samples_leaf': randint(1, 10),     
    'simpleimputer__strategy': ['mean', 'median'], 
    'randomforestregressor__n_estimators': randint(50, 500), 
    'randomforestregressor__max_depth': [5, 10, 15, 20, None], 
    'randomforestregressor__max_features': uniform(0, 1) # max_features
}

clf = RandomizedSearchCV(
    pipe, 
    param_distributions=dists, 
    n_iter=50, 
    cv=3, 
    scoring='neg_mean_absolute_error',  
    verbose=1,
    n_jobs=-1
)

clf.fit(X_train, y_train);

Fitting 3 folds for each of 50 candidates, totalling 150 fits


/usr/local/lib/python3.7/dist-packages/category_encoders/target_encoder.py:94: FutureWarning: Default parameter min_samples_leaf will change in version 2.6.See https://github.com/scikit-learn-contrib/category_encoders/issues/327
  category=FutureWarning)
/usr/local/lib/python3.7/dist-packages/category_encoders/target_encoder.py:99: FutureWarning: Default parameter smoothing will change in version 2.6.See https://github.com/scikit-learn-contrib/category_encoders/issues/327
  category=FutureWarning)


In [55]:
print('최적 하이퍼파라미터: ', clf.best_params_)
print('MAE: ', -clf.best_score_)

최적 하이퍼파라미터:  {'randomforestregressor__max_depth': 20, 'randomforestregressor__max_features': 0.1898463906093224, 'randomforestregressor__n_estimators': 304, 'simpleimputer__strategy': 'median', 'targetencoder__min_samples_leaf': 3, 'targetencoder__smoothing': 1000.0}
MAE:  15672.099682990482


In [56]:
pd.DataFrame(clf.cv_results_).sort_values(by='rank_test_score').T


,18,42,21,0,43,32,10,1,38,3,...,7,20,22,27,30,35,41,11,12,4
mean_fit_time,5.169643,2.844576,4.494326,2.431127,3.96847,4.649462,2.980283,3.520257,2.624512,5.798872,...,2.803648,4.487976,8.145532,4.25304,4.088235,2.865008,2.096611,4.677804,1.931009,2.160998
std_fit_time,0.394759,0.056414,0.117832,0.32907,0.032487,0.693934,0.019799,0.045563,0.047314,0.021946,...,0.011047,0.685149,0.464838,0.215891,0.190911,0.271817,0.019824,0.019082,0.017257,0.082771
mean_score_time,0.398664,0.22367,0.401257,0.191228,0.288964,0.327123,0.235536,0.282151,0.231275,0.293221,...,0.280017,0.321565,0.492404,0.364801,0.450531,0.207525,0.229812,0.256966,0.235502,0.246323
std_score_time,0.014844,0.003466,0.034044,0.007432,0.016848,0.098499,0.009622,0.003811,0.023472,0.007358,...,0.004431,0.055893,0.080002,0.03073,0.021097,0.012443,0.008381,0.010436,0.01064,0.018112
param_randomforestregressor__max_depth,20,15,15,15,10,20,20,15,10,None,...,5,5,5,5,5,5,5,5,10,5
param_randomforestregressor__max_features,0.189846,0.315954,0.367585,0.374228,0.30379,0.383645,0.29563,0.170644,0.227228,0.494106,...,0.176956,0.64169,0.843105,0.808992,0.933984,0.985113,0.100896,0.992816,0.018797,0.039493
param_randomforestregressor__n_estimators,304,227,206,104,444,352,255,428,260,451,...,474,191,463,208,116,196,325,445,297,404
param_simpleimputer__strategy,median,median,median,mean,median,mean,median,median,median,mean,...,median,mean,mean,median,median,median,median,mean,mean,mean
param_targetencoder__min_samples_leaf,3,4,9,9,5,7,1,3,6,4,...,8,4,3,6,1,9,3,1,7,7
param_targetencoder__smoothing,1000.0,500.0,500.0,50.0,2.0,100.0,60.0,50.0,60.0,1000.0,...,100.0,50.0,60.0,20.0,50.0,1000.0,2.0,50.0,500.0,2.0


In [57]:
pipe = clf.best_estimator_

In [58]:
from sklearn.metrics import mean_absolute_error

y_pred = pipe.predict(X_test)
mae = mean_absolute_error(y_test, y_pred)
print(f'테스트세트 MAE: ${mae:,.0f}')

테스트세트 MAE: $16,010


In [59]:
features

Index(['MSSubClass', 'MSZoning', 'LotFrontage', 'LotArea', 'Street', 'Alley',
       'LotShape', 'LandContour', 'Utilities', 'LotConfig', 'LandSlope',
       'Neighborhood', 'Condition1', 'Condition2', 'BldgType', 'HouseStyle',
       'OverallQual', 'OverallCond', 'YearBuilt', 'YearRemodAdd', 'RoofStyle',
       'RoofMatl', 'Exterior1st', 'Exterior2nd', 'MasVnrType', 'MasVnrArea',
       'ExterQual', 'ExterCond', 'Foundation', 'BsmtQual', 'BsmtCond',
       'BsmtExposure', 'BsmtFinType1', 'BsmtFinSF1', 'BsmtFinType2',
       'BsmtFinSF2', 'BsmtUnfSF', 'TotalBsmtSF', 'Heating', 'HeatingQC',
       'CentralAir', 'Electrical', 'BsmtFullBath', 'BsmtHalfBath', 'FullBath',
       'HalfBath', 'BedroomAbvGr', 'KitchenAbvGr', 'KitchenQual',
       'TotRmsAbvGrd', 'Functional', 'Fireplaces', 'FireplaceQu', 'GarageType',
       'GarageYrBlt', 'GarageFinish', 'GarageCars', 'GarageArea', 'GarageQual',
       'GarageCond', 'PavedDrive', 'WoodDeckSF', 'OpenPorchSF',
       'EnclosedPorch', '3SsnPorch